In [ ]:
# CODE BY PRINCE

# Create database named bank before executing the code

#Importing psycopg2 and making connection

import psycopg2
import getpass

mydb = psycopg2.connect(database = 'bank',
                       user = 'postgres',
                       password ='*****',         # enter your password
                       host = 'localhost',
                       port = '5432')
mycursor = mydb.cursor()

#creating table aadhar_details

mycursor.execute("create table aadhar_details (name varchar(50) not null,\
                 phone_number bigint not null,\
                 email_id varchar(50) unique,\
                 aadhar_number bigint primary key)")
mydb.commit()

#creating table bank_accounts

mycursor.execute("create sequence an_seq start with 10000001")
mycursor.execute("create table bank_accounts (name varchar(50),\
                 phone_number bigint not null,\
                 aadhar_number bigint ,\
                 account_type varchar(30),\
                 account_number varchar(30) not null default 'an_'||nextval('an_seq'::regclass) ,\
                 password varchar(30),\
                 time_created timestamp default now(),\
                 primary key (account_number),\
                 constraint fk_aadhar_details\
                    foreign key (aadhar_number)\
                    references aadhar_details (aadhar_number) ) ")
mydb.commit()

# Creating table for each account
    
mycursor.execute('create or replace function table_creation_func() returns trigger as \
                    $body$\
                    declare\
                        ac_num varchar(20);\
                    begin\
                        select account_number into ac_num from bank_accounts where  aadhar_number = new.aadhar_number  and account_type = new.account_type ;\
                        execute format (\'create table %s (account_number varchar(30) ,\
                                                           credit int default 0,\
                                                           debit int default 0,\
                                                           available_balance bigint,\
                                                           transacted_time timestamp default now())\',ac_num);\
                                                           return new;\
                    end;\
                    $body$\
                    language plpgsql')
                 
mycursor.execute('create trigger table_creation_trig\
                      after insert on bank_accounts\
                      for each row\
                      execute procedure table_creation_func();')

mydb.commit()

In [ ]:
#Importing psycopg2 and making connection

import psycopg2
import getpass
import tabulate

mydb = psycopg2.connect(database = 'bank',
                       user = 'postgres',
                       password ='*****',              #enter your password
                       host = 'localhost',
                       port = '5432')
mycursor = mydb.cursor()

# option to create, login and deposit

def choose():
    x = int(input("1. Create new account\t2. Log in\t3. Deposit\t4. Exit\t"))
    if x == 1:
        newaccount()
    elif x == 2:
        login()
    elif x == 3:
        deposit()
    elif x == 4:
        exit()
    else:
        print("Invalid input")
        choose()

# creating new account

def newaccount():
    
    def phcheck(n):
        if n//10e8 in [6,7,8,9]:
            return True 

    def anumcheck(n):
        count = 0
        while n != 0:
            n //= 10
            count += 1
        if count == 12:
            return True
    
    def aadhar():
        
        #feeding rows in aadhar card table
    
        name = input("\nEnter your name : ")
        ph = int(input("Enter your phone number : "))
        mail = input("Enter your Email_id : ")
        anum = int(input("Enter your Aadhar number : "))
        inputs = """insert into aadhar_details values (%s,%s,%s,%s)"""
        aput = """select aadhar_number from aadhar_details where aadhar_number = %s"""
        mycursor.execute(aput,(anum,))
        x = mycursor.fetchone()
        if x is None:
            if phcheck(ph) == anumcheck(anum) == True : 
                mycursor.execute(inputs,(name,ph,mail,anum))
                mydb.commit()
                print("\n...\n")
                print("Aadhar details successfully updated.")             
            else:
                print("\nInvalid Phone number or Aadhar number\n")
                t_g = int(input("1. Try again\t2. Exit"))
                if t_g == 1:
                    name,ph,anum = aadhar()
                elif t_g == 2:
                    exit()
                else:
                    print("Invalid input")
                    exit()
        else:
            print("\nAadhar ID already exists . \nTry with new Aadhar ID or contact bank to proceed with the same.")
            t_g = int(input("\n1. Try again\t2. Exit"))
            if t_g == 1:
                name,ph,anum = aadhar()
            elif t_g == 2:
                exit()
            else:
                print("Invalid input")
                exit()
    
        return name,ph,anum
        
    def chooseaccount():
        
        ac_types = ['Savings account', 'Current Account', 'Salary Account', 'Fixed Deposit Account']
        acc_type = int(input("\n1. Savings Account\t2. Current Account\t3. Salary Account\t4. Fixed Deposit "))
        if acc_type in [1,2,3,4]:
            a_type = ac_types[acc_type-1]
        else:
            print("\nInvalid option")
            a_type = chooseaccount()
        return a_type
    
    def password():
        
        p_word = getpass.getpass("\nEnter password : ")
        cp_word = getpass.getpass("Confirm password : ")
        if p_word == cp_word:
            return p_word     
        else:
            print("\nPasswords do not match")
            p_word = password()
        return p_word
        
    print("\nSTEP 1 : UPLOAD AADHAR DETAILS")
    name,ph,anum = aadhar()
    print("\nSTEP 2 : CHOOSE ACCOUNT TYPE")
    a_type = chooseaccount()
    print("\nSTEP 3 : SET PASSWORD")
    p_word = password()
    inputs = """insert into bank_accounts (name,phone_number,aadhar_number,account_type,password) values (%s,%s,%s,%s,%s)"""
    mycursor.execute(inputs,(name,ph,anum,a_type,p_word))
    mydb.commit()
    print("\n...\n")
    print("Congradulations ! You have successfully created a ",a_type)
    
    an_get = """select account_number from bank_accounts where aadhar_number = %s and account_type = %s"""
    mycursor.execute(an_get,(anum,a_type,))
    x = mycursor.fetchone()
    print("\nYour account type   : ",a_type)
    print("Your account number : ",x[0])
    
    print("\nInitially you have to deposit minimum amount in your account")
    uan = input("\nEnter your account number : ")
    ini_amt = int(input("\nEnter the initial amount to deposit : \nIn multiples of 100 : "))
    
    ini_amt_put = """insert into {} (account_number,credit,available_balance) values ('{}',{},{})"""
    mycursor.execute(ini_amt_put.format(uan,uan,ini_amt,ini_amt))
   
    mydb.commit()
    
    print("\nAmount deposited : ",ini_amt)
    print("Available balance : ",ini_amt)
    

def balance_enquiry(user_acnum):
  
    bal_st = """select available_balance from {} where ctid = (select max(ctid) FROM {}) """
    mycursor.execute (bal_st.format(user_acnum,user_acnum))
    x = mycursor.fetchone()
    print("\nAvailable balance : ",x[0])   
        
    
# Logging into account 

def login():
    
    def withdraw():
        amount = int(input("\nEnter the amount : "))
        bal_ch = """select available_balance from {} where account_number = '{}' """
        mycursor.execute (bal_ch.format(user_acnum,user_acnum))
        x = mycursor.fetchone()
        if int(x[0])>=amount:
            print("\nTransaction complete ")
            get = """select available_balance from {} where ctid = (select max(ctid) FROM {})"""
            mycursor.execute(get.format(user_acnum,user_acnum))
            a_bal = mycursor.fetchone()
            r_bal = a_bal[0]-amount
            feed = """insert into {} (account_number,debit,available_balance) values ('{}',{},{})"""
            mycursor.execute(feed.format(user_acnum,user_acnum,amount,r_bal))
            mydb.commit()
            print("Amount debited : ",amount)
            balance_enquiry(user_acnum)
            
            
        else:
            print("\nInsufficient balance")
            
    def viewstatement():
        v_st = """select * from {}"""
        mycursor.execute(v_st.format(user_acnum))
        statement = mycursor.fetchall()
        from tabulate import tabulate as tab
        print("\n")
        print ("{:>55}".format("Transaction details"))
        print(tab(statement,headers = [i[0] for i in mycursor.description], tablefmt = 'psql'))
                             
    def purpose():
        
        purpose = int(input("\n1. Withdraw\t2. Balance Enquiry\t3. View statement "))
        if purpose == 1:
            withdraw()
        elif purpose == 2:
            balance_enquiry(user_acnum)
        elif purpose == 3:
            viewstatement()
        else:
            print("\nInvalid option")
            purpose()
        
    
    user_acnum = input("\nEnter account number : ")
    user_password = input("Enter password : ")
    check = """select password from bank_accounts where account_number = %s """
    mycursor.execute (check,(user_acnum,))
    x = mycursor.fetchone()
    if x is not None and x[0]==user_password :
        purpose()      
    else:
        print("\nInvalid Account number or password")
        
# depositing into account

def deposit():
    
    user_acnum = input("\nEnter the account number : ")
    amount = int(input("Enter the amount to be deposited : "))
    get = """select available_balance from {} where ctid = (select max(ctid) FROM {})"""
    mycursor.execute(get.format(user_acnum,user_acnum))
    a_bal = mycursor.fetchone()
    r_bal = a_bal[0]+amount
    
    upt = """insert into {}(account_number,credit,available_balance) values ('{}',{},{})"""
    mycursor.execute(upt.format(user_acnum,user_acnum,amount,r_bal))
    mydb.commit()
    
    print("\nAmount deposited : ",amount)
    balance_enquiry(user_acnum)




# calling options
choose()